# Programación 201

Librerías científicas en Python: `Pandas`
------------------------------------------------------

### Ejercicio 1

Cargad los datos del fichero _got.csv_ en un _dataframe_. Este conjunto de datos recoge información de la [Guerra de los Cinco Reyes](https://awoiaf.westeros.org/index.php/War_of_the_Five_Kings) de las novelas de [Cancion de Hielo y Fuego](https://es.wikipedia.org/wiki/Canci%C3%B3n_de_hielo_y_fuego) de George R.R Martin.

Mostrad el número de filas del _dataframe_ y las etiquetas de los ejes.

In [18]:
# Respuesta
import pandas
dataFile = '..\..\..\dataset\got.csv'
df = pandas.read_csv(dataFile)
rows = df.shape[0] 
print(f'El dataframe tiene {rows} filas.')
tags = [e for e in df.axes[1]]
# o en alternativa
tags = [e for e in df.columns]
print('Estas son las etiquetas de los ejes:')
print(tags)

El dataframe tiene 38 filas.
Estas son las etiquetas de los ejes:
['name', 'year', 'battle_number', 'attacker_king', 'defender_king', 'attacker_1', 'attacker_2', 'attacker_3', 'attacker_4', 'defender_1', 'defender_2', 'defender_3', 'defender_4', 'attacker_outcome', 'battle_type', 'major_death', 'major_capture', 'attacker_size', 'defender_size', 'attacker_commander', 'defender_commander', 'summer', 'location', 'region', 'note']


### Ejercicio 2

Agrupad los datos cargados en el ejercicio 1 por el principal bando atacante (*attacker\_1*). Para cada posición, mostrad el número de batallas y el resultado de la batalla (el resultado se encuentra en el campo *attacker\_outcome*).

In [19]:
# Respuesta
df[['attacker_1', 'battle_number', 'attacker_outcome']].sort_values(by='attacker_1')
# Groupby es un generados
df2 = df.groupby(by='attacker_1')['attacker_outcome', 'battle_number'].count()


<ipython-input-19-5039b1644e0c>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = df.groupby(by='attacker_1')['attacker_outcome', 'battle_number'].count()


In [20]:
df3 = df.groupby(by='attacker_1', squeeze=True)
df3 = df3['attacker_outcome'].value_counts()

<ipython-input-20-0adec4275f87>:1: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  df3 = df.groupby(by='attacker_1', squeeze=True)


In [21]:
for (attacker, battles) in df3.groupby(['attacker_1']):
    print(f'El número de batallas realizadas por {attacker} fueron {len(battles)}')

El número de batallas realizadas por Baratheon fueron 2
El número de batallas realizadas por Bolton fueron 1
El número de batallas realizadas por Bracken fueron 1
El número de batallas realizadas por Brave Companions fueron 1
El número de batallas realizadas por Brotherhood without Banners fueron 1
El número de batallas realizadas por Darry fueron 1
El número de batallas realizadas por Free folk fueron 1
El número de batallas realizadas por Frey fueron 1
El número de batallas realizadas por Greyjoy fueron 1
El número de batallas realizadas por Lannister fueron 2
El número de batallas realizadas por Stark fueron 2


### Ejercicio 3

Mostrad los datos de las batallas donde el numero de participantes supera los 15000 "hombres" (contando ambos bandos: *attacker\_size* y *defender\_size*), el resultado haya sido favorable para el atacante, y la batalla se haya producido en invierno (_summer=0_).

In [29]:
# Respuesta
df[df.attacker_size + df.defender_size >= 15000][df.summer == 0][df.attacker_outcome == 'win']
# Alternativa
df[(df.attacker_size + df.defender_size >= 15000)&(
    df.summer == 0)&(
        df.attacker_outcome == 'win')]

<ipython-input-29-f2d8fa9a0a0e>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.attacker_size + df.defender_size >= 15000][df.summer == 0][df.attacker_outcome == 'win']


,name,year,battle_number,attacker_king,defender_king,attacker_1,attacker_2,attacker_3,attacker_4,defender_1,...,major_death,major_capture,attacker_size,defender_size,attacker_commander,defender_commander,summer,location,region,note


### Ejercicio 4

Contad el número de lugares que aparecen más de una vez en _dataframe_ (campo _location_), utilizando las funciones de la librería _pandas_. ¿Existe algún lugar donde se haya realizado más de una batalla? Comprobad qué bandos estaban implicados.

In [30]:
# Respuesta
df['location'].value_counts

<bound method IndexOpsMixin.value_counts of 0                             Golden Tooth
1                            Mummer's Ford
2                                 Riverrun
3                               Green Fork
4                          Whispering Wood
5                                 Riverrun
6                                    Darry
7                              Moat Cailin
8                           Deepwood Motte
9                              Stony Shore
10                        Torrhen's Square
11                              Winterfell
12                        Torrhen's Square
13                              Winterfell
14                                 Oxcross
15                             Storm's End
16                                Red Fork
17                               Harrenhal
18                                    Crag
19                          King's Landing
20                                   Darry
21                              Duskendale
22        

### Ejercicio 5

Añadid una nueva columna al _dataframe_ con un valor booleano indicando una predicción básica de si en una cierta batalla el bando atacante será vencedor. Definiremos el valor como _True_ sí el bando atacante es mayor al defensor, y como _False_ en caso contrario. Definid como _NaN_ aquellos casos en los que no haya número de tropas en ningún bando. ¿En que casos, identificados por el número de batalla (*battle\_number*), nuestra predicción coincide con el resultado real?

In [24]:
# Respuesta
# pandas apply y trasform

# Solución Sebastian
import numpy as np
conditions = [
    (df['attacker_size'] > df['defender_size']),
    (df['attacker_size'] < df['defender_size'])]
choices = [True, False]
df['prediction'] = np.select(conditions, choices, default='NaN')

conditions2 = [
    ((df['prediction'] == True) & (df['attacker_outcome'] == 'win')),
    ((df['prediction'] == True) & (df['attacker_outcome'] == 'loss')),
    ((df['prediction'] == False) & (df['attacker_outcome'] == 'loss')),
    ((df['prediction'] == False) & (df['attacker_outcome'] == 'win'))]
choices2 = [True, False, True, False]
df['succesful_prediction'] = np.select(conditions, choices, default='NaN')

df['succesful_prediction'].value_counts()
